In [ ]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
listings_file = "listings.csv"
listings_df = pd.read_csv(listings_file)
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1947,Furnished Gold Coast studio #806,2153,Linda,NaN,Near North Side,41.90246,-87.63405,Entire home/apt,59,14,4,3/29/2020,0.06,7,3
1,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,55,2,178,12/15/2019,2.76,1,322
2,4505,Open to healthcare professionals only until Ju...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,117,2,394,2/16/2020,2.91,1,146
3,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,80,2,384,3/8/2020,2.92,1,335
4,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,150,4,49,10/23/2019,0.67,11,320


In [3]:
neighborhood_file = "Neighborhood_Zhvi_AllHomes.csv"
neighborhood_df = pd.read_csv(neighborhood_file)
neighborhood_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/1996,...,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020
0,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,134197.0,...,328763.0,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0
1,112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,...,184177.0,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0
2,192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,140042.0,...,268101.0,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0
3,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,247356.0,...,1245653.0,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0
4,118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,135240.0,...,509245.0,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0


### Transform Listings DataFrame

In [4]:
# Create a filtered dataframe from specific columns
listings_cols = ['id','name','host_id','host_name','neighbourhood','latitude','longitude','room_type','price','minimum_nights',
'number_of_reviews','last_review','reviews_per_month','calculated_host_listings_count','availability_365']
listings_transformed= listings_df[listings_cols].copy()


# Rename the column headers
listings_transformed = listings_transformed.rename(columns={"neighbourhood": "neighborhood"})

# Clean the data by dropping duplicates and setting the index
listings_transformed.drop_duplicates("id", inplace=True)
listings_transformed.set_index("id", inplace=True)

listings_transformed.head()

,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,
1947,Furnished Gold Coast studio #806,2153,Linda,Near North Side,41.90246,-87.63405,Entire home/apt,59,14,4,3/29/2020,0.06,7,3
2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,55,2,178,12/15/2019,2.76,1,322
4505,Open to healthcare professionals only until Ju...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,117,2,394,2/16/2020,2.91,1,146
7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,80,2,384,3/8/2020,2.92,1,335
9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,150,4,49,10/23/2019,0.67,11,320


### Transform Neighborhood DataFrame

In [5]:
neighborhood_cols = ['RegionID','SizeRank','RegionName','State','City','Metro','CountyName','1/31/2019','2/28/2019',
'3/31/2019','4/30/2019','5/31/2019','6/30/2019','7/31/2019','8/31/2019','9/30/2019','10/31/2019','11/30/2019','12/31/2019',
'1/31/2020','2/29/2020','3/31/2020','4/30/2020']
neighborhood_transformed = neighborhood_df[neighborhood_cols].copy()



# Rename the column headers
neighborhood_transformed = neighborhood_transformed.rename(columns={"RegionID":"ID","RegionName":"Neighborhood"})

neighborhood_transformed.drop_duplicates("ID", inplace=True)

# Set index
neighborhood_transformed.set_index("ID", inplace=True)

neighborhood_transformed.head()

,SizeRank,Neighborhood,State,City,Metro,CountyName,1/31/2019,2/28/2019,3/31/2019,4/30/2019,...,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020
ID,,,,,,,,,,,,,,,,,,,,,
274772,0,Northeast Dallas,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,324637.0,326062.0,327092.0,327497.0,...,328763.0,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0
112345,1,Maryvale,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,179304.0,180222.0,181021.0,181837.0,...,184177.0,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0
192689,2,Paradise,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,267475.0,268947.0,269033.0,269480.0,...,268101.0,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0
270958,3,Upper West Side,NY,New York,New York-Newark-Jersey City,New York County,1290079.0,1278523.0,1280139.0,1266046.0,...,1245653.0,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0
118208,4,South Los Angeles,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,501378.0,501129.0,500697.0,501828.0,...,509245.0,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0


In [6]:
neighborhood_transformed["avgprice"]=""

In [7]:
neighborhood_transformed["avgprice"]=round(neighborhood_transformed.iloc[:,6:22].mean(axis=1), 2)

In [8]:
neighborhood_cook=neighborhood_transformed.loc[(neighborhood_transformed["State"] == "IL") &
                            (neighborhood_transformed["CountyName"] == "Cook County")]

In [9]:
neighborhood_transformed.head()

,SizeRank,Neighborhood,State,City,Metro,CountyName,1/31/2019,2/28/2019,3/31/2019,4/30/2019,...,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,avgprice
ID,,,,,,,,,,,,,,,,,,,,,
274772,0,Northeast Dallas,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,324637.0,326062.0,327092.0,327497.0,...,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0,329124.81
112345,1,Maryvale,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,179304.0,180222.0,181021.0,181837.0,...,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0,187454.75
192689,2,Paradise,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,267475.0,268947.0,269033.0,269480.0,...,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0,269702.75
270958,3,Upper West Side,NY,New York,New York-Newark-Jersey City,New York County,1290079.0,1278523.0,1280139.0,1266046.0,...,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0,1239603.81
118208,4,South Los Angeles,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,501378.0,501129.0,500697.0,501828.0,...,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0,516624.44


In [10]:
neighborhood_cook.rename(columns={"Neighborhood":"neighborhood"}, inplace=True)

C:\Users\pli.TELOS\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [22]:
#merge df
merged_df= pd.merge(listings_transformed, neighborhood_cook, on="neighborhood",left_index=True)

# merged_df=merged_df.drop(merged_df.columns[21:36], axis=1)

merged_df.keys()


merged_df=merged_df.drop(['1/31/2019', '2/28/2019', '3/31/2019', '4/30/2019', '5/31/2019',
       '6/30/2019', '7/31/2019', '8/31/2019', '9/30/2019', '10/31/2019',
       '11/30/2019', '12/31/2019', '1/31/2020', '2/29/2020', '3/31/2020',
       '4/30/2020'], axis=1)


# merged_df = merged_df.rename(columns={"id":'ID'})

# merged_df = merged_df.Index.rename('id')
merged_df.rename(index={"id":"ID"})


merged_df



,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,SizeRank,State,City,Metro,CountyName,avgprice
ID,,,,,,,,,,,,,,,,,,,,
269586,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,55,2,178,12/15/2019,2.76,1,322,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,35,2,44,2/14/2020,0.66,5,268,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Historic Chicago Home - Hyde Park,1517871,Jon,Hyde Park,41.80086,-87.58948,Entire home/apt,195,7,8,9/17/2018,0.08,2,348,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Spacious Stylish One Bedroom Apt In HydePark,9192868,Rebecca,Hyde Park,41.79984,-87.59465,Entire home/apt,101,2,337,3/10/2020,4.25,1,332,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Musician's Quarters,9601147,Chester,Hyde Park,41.79963,-87.59477,Entire home/apt,115,3,45,8/5/2019,0.59,1,329,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137634,Akounamatata,302616563,Ousmane,Burnside,41.72621,-87.59435,Entire home/apt,50,1,9,2/17/2020,2.21,1,39,1883,IL,Chicago,Chicago-Naperville-Elgin,Cook County,123820.50
137634,Cozy Bachelor Pad 10 Minutes From Downtown.,278029467,Eli,Burnside,41.72479,-87.59414,Private room,95,14,0,NaN,NaN,1,180,1883,IL,Chicago,Chicago-Naperville-Elgin,Cook County,123820.50
275192,Front Room,314605732,GrapefruitF,Riverdale,41.65208,-87.61030,Private room,20,3,0,NaN,NaN,1,0,15654,IL,Chicago,Chicago-Naperville-Elgin,Cook County,56633.88


### Create database connection

In [23]:
connection_string = "postgres:Bootcamp2017@localhost:5432/Airbnb"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
# Confirm tables
engine.table_names()

['listing', 'neighborhood', 'listing_neighborhood']

### Load DataFrames into database

In [14]:
listings_transformed.to_sql(name='listing', con=engine, if_exists='append', index=True)

In [15]:
neighborhood_transformed.to_sql(name='neighborhood', con=engine, if_exists='append', index=True)

In [25]:
merged_df.to_sql(name='listing_neighborhood', con=engine, if_exists='append', index=True)